In [4]:
using Pkg; Pkg.activate(".");
using Convex
using SCS
c = 5*rand(7)

  Activating new project at `~/Convex_Opt/6.S098-project`


7-element Vector{Float64}:
 3.7049990645092103
 3.9830928127219307
 1.9493424287429613
 1.2289982600147402
 0.8028347571174799
 4.220466418355666
 4.677294408903271

In [4]:
c = 5*rand(7)

x = Variable(7)
objective = c'*x

cons = [
    ones(7)'*x <= 4,
    x >= 0,
    x <= 1
]



optimization = maximize(objective, cons)
solve!(optimization, SCS.Optimizer())
res = x.value
print(c'*res)

res





[12.234156867844309;;]------------------------------------------------------------------
	       SCS v3.2.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 8, constraints m: 16
cones: 	  z: primal zero / dual free vars: 1
	  l: linear vars: 15
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 29, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 4.00e+01  4.37e+00  1.12e+02 -7.07e+01  1.00e-01  1.98e-03 
    50| 9.69e-05  6.55e-06  2.45e-06 -1.22e+01  1.00e-01  2.03e-03 
-----------------

7×1 Matrix{Float64}:
  9.6878600669219e-5
  0.9999921697655961
  1.000002147569336
  4.495997615960186e-6
 -3.82716739263455e-5
  0.9999719740161574
  0.9999719321672004

In [10]:
###Trial
class_numbers = Dict("6.006"=>1, "6.042"=>2, "6.009"=>3, "6.046"=>4, "6.031"=>5, "18.600"=>6, "6.041"=>7,
"6.008"=>8, "comp_bio"=>9, "NLP"=>10, "comp_vision"=>11, "graphics"=>12, "optimization"=>13, 
    "script_analysis"=>14, "intro_to_acting"=>15, "psych"=>16)

#TODO: EXPAND TO 3 SEM LEFT, GET EXAMPLES THAT ARE INFEASIBLE AND PLAY WITH PENALTY LAMBDA

#THIS TAKEN WITH 50 HR CONSTRAINT IS INFEASIBLE
taken = [1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0]

requirements = [([1], 1), ([2], 1), ([3], 1), ([4], 1), ([5], 1), ([6, 7, 8], 1), ([9, 10, 11, 12, 13], 2), 
    ([14, 15, 16], 2)]

remaining_reqs = Dict()

for (ind, req) in enumerate(requirements)
    remaining = req[2]
    for class in req[1]
        if taken[class] == 1
            remaining -= 1
            if remaining == 0
                break
            end
            
        end
        
    end
    remaining_reqs[ind] = remaining
end

remaining_reqs





Dict{Any, Any} with 8 entries:
  5 => 1
  4 => 1
  6 => 0
  7 => 2
  2 => 0
  8 => 2
  3 => 1
  1 => 0

In [13]:
c = 7*rand(16)
time_vec = 7*ones(16) + 7*rand(16)
c

16-element Vector{Float64}:
 1.1346630484277376
 6.767848071180939
 6.195504950689403
 1.094820691597862
 0.738777330623374
 3.28325569570889
 3.857255914650558
 4.463000565589705
 5.971209186206194
 0.04041376305274802
 5.992987678483452
 5.61490864894082
 6.2083155222320014
 5.104620764595033
 3.2901542452686146
 0.9042647381711935

In [14]:
x = Variable(16)
objective = c'*x - 3*sum(x)
t = 50
constr = [
    x >= 0,
    x <= 1,
    time_vec'*x <= t
    
]
for (class, status) in enumerate(taken)
    
    if status == 1
        
        push!(constr, (x[class] == 0))    
    end   
end


for (ind, req) in enumerate(requirements)
    length = size(req[1])[1]
    
    req_array = Array{Convex.AbstractExpr}(undef, length)
    for i in 1:length[1]
        req_array[i] = x[req[1][i]]
        
    end
    push!(constr, (sum(req_array) >= remaining_reqs[ind]))
end


optimization = maximize(objective, constr)
solve!(optimization, SCS.Optimizer())
res = x.value
print(c'*res)

res





[NaN;;]------------------------------------------------------------------
	       SCS v3.2.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 17, constraints m: 45
cones: 	  z: primal zero / dual free vars: 4
	  l: linear vars: 41
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 84, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 1.03e+02  4.27e+00  8.31e+01 -4.71e+01  1.00e-01  1.35e-04 
    75| 9.51e+11  1.34e+10  5.71e+19  2.86e+19  1.00e-01  2.69e-04 
-------------------------------

┌ Warning: Problem status INFEASIBLE; solution may be inaccurate.
└ @ Convex /Users/andrewhutchison/.julia/packages/Convex/ukggP/src/solution.jl:342


16×1 Matrix{Float64}:
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN

In [14]:
final_vector = round.(res, digits = 3)

16×1 Matrix{Float64}:
  0.0
 -0.0
  0.0
  1.0
  1.0
 -0.0
  1.0
 -0.0
  1.0
  1.0
  1.0
 -0.0
  1.0
 -0.0
  1.0
 -0.0

In [77]:
time_vec[4] + time_vec[5] + time_vec[9] + time_vec[11] + time_vec[15]

54.60584579911304

In [78]:
time_vec'*res

1×1 adjoint(::Vector{Float64}) with eltype Float64:
 49.998805938308834